# sprint23課題　深層学習スクラッチリカレントニューラルネットワーク

## 【問題1】SimpleRNNのフォワードプロパゲーション実装

SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。  

今回はバッチサイズをbatch_size、入力の特徴量数をn_features、RNNのノード数をn_nodesとして表記します。活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。  

フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。  


$$ a_t = x_{t}\cdot W_{x} + h_{t-1}\cdot W_{h} + b\\ $$
$$h_t = tanh(a_t)$$


$a_t$ : 時刻tの活性化関数を通す前の状態 (batch_size, n_nodes)  
$h_t$ : 時刻tの状態・出力 (batch_size, n_nodes)  
$x_t$ : 時刻tの入力 (batch_size, n_features)  
$W_x$ : 入力に対する重み (n_features, n_nodes)  
$h_t−1$ : 時刻t-1の状態（前の時刻から伝わる順伝播） (batch_size, n_nodes)  
$W_h$ : 状態に対する重み。 (n_nodes, n_nodes) 　  
$b$ : バイアス項 (1,)  
初期状態 $h_0$ は全て0とすることが多いですが、任意の値を与えることも可能です。

上記の処理を系列数n_sequences回繰り返すことになります。RNN全体への入力 
x
は(batch_size, n_sequences, n_features)のような配列で渡されることになり、そこから各時刻の配列を取り出していきます。

分類問題であれば、それぞれの時刻のhに対して全結合層とソフトマックス関数（またはシグモイド関数）を使用します。出力は最後のhだけを使用する場合と、全てのhを使う場合があります。


In [ ]:
class SimpleRNN:
    """
    ノード数pre_nodesからnodesへの全結合層
    Parameters
    ----------
    nodes : int
      層のノード数
    
    Attributes
    ----------
    optimizer : 最適化のクラス
    W : 重み
    B : バイアス
    dW : 重みの勾配
    dB : バイアスの勾配    
    """
    def __init__(self, nodes):
        self.nodes = nodes
        self.group = 'RNN'
        
    def initialize(self, feature, initializer, optimizer, sigma=1e-2, lr=1e-2):
        """
        重み、バイアスを初期化して出力数を渡してあげる
        Parameters
        ----------
        input_dim :次の形のtuple, (入力チャンネル,高さ,横幅)
        initializer: class
        optimizer: class
        lr : float(1e-2)
            optimizerに渡す学習率
        sigma : float(1e-2)
            Simpleinitializerを選んだ時のパラメータ
        """
        
        #初期値を設定する。
        if initializer != SimpleInitializer:
            initializer = initializer()
            self.W_x = initializer.W(feature , feature, self.nodes)
            self.W_h = initializer.W(feature , self.nodes, self.nodes)
            self.B = np.array([1])
        else:
            initializer = initializer(sigma)
            self.W_x = initializer.W(feature, self.nodes)
            self.W_h = initializer.W(self.nodes, self.nodes)
            self.B = np.array([1])
        
        #optimizerを設定する。
        self.optimizer = optimizer(lr)
        
        return self.nodes

    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, pre_nodes)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, nodes)
            出力
        """        
        self.Z = X.copy()
        A = X @ self.W + self.B
        return A
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, nodes)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, pre_nodes)
            前に流す勾配
        """
        self.dB = dA
        self.dW = self.Z.T @ dA
        dZ = dA @ self.W.T
        # 更新
        self = self.optimizer.update(self)
        return dZ


## 【問題2】小さな配列でのフォワードプロパゲーションの実験

In [21]:
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes))
b = np.array([1])

In [22]:
act = Tanh()
for t in range(n_sequences):
    a = x[:, t, :] @ w_x + h @ w_h + b
    h = act.forward(a)

In [23]:
h

array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])

In [16]:
a_0 = x[:,0,:] @ w_x + b
act1 = Tanh()
h = act1.forward(a_0)
h

array([[0.76188798, 0.76213958, 0.76239095, 0.76255841]])

In [20]:
a_1 = x[:,1,:] @ w_x + h @ w_h + b
h = act1.forward(a_1)
h

array([[0.792209  , 0.8141834 , 0.83404912, 0.84977719]])

In [4]:
class Tanh:
    """
    ハイパボリックタンジェント関数
    Parameters
    ----------
    """
    def __init__(self):
        self.Z = None
        self.group = 'activation'

    def forward(self, A):
        """
        フォワードプロパゲーションのときのメソッド
        Parameters
        ----------
        A : 全結合後の行列 shape(batch_size, n_nodes2)

        Returns
        ----------
        Z : 活性化後の行列　shape(batch_size, n_nodes2)
        """
        self.Z = np.tanh(A)
        return self.Z
    
    def backward(self, dZ):
        """
        バックワード
        Parameters
        ----------
        dZ : 全結合後の行列 shape(batch_size, n_nodes2)

        Returns
        ----------
        dA : 活性化後の行列　shape(batch_size, n_nodes2)
        """
        dA = dZ * (1 - self.Z ** 2)
        return dA